In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
df_movies = pd.read_csv('../Data/raw/movies/the-movies-dataset/movies_metadata.csv', 
                        low_memory=False)

In [3]:
df_movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
adult                    45466 non-null object
belongs_to_collection    4494 non-null object
budget                   45466 non-null object
genres                   45466 non-null object
homepage                 7782 non-null object
id                       45466 non-null object
imdb_id                  45449 non-null object
original_language        45455 non-null object
original_title           45466 non-null object
overview                 44512 non-null object
popularity               45461 non-null object
poster_path              45080 non-null object
production_companies     45463 non-null object
production_countries     45463 non-null object
release_date             45379 non-null object
revenue                  45460 non-null float64
runtime                  45203 non-null float64
spoken_languages         45460 non-null object
status                   45379 non-null objec

Firstly, we note that:

1. There are some columns useless for our analysis that we could dropping (`poster_path`, `homepage`).
2. Movie `id` are objects, not integers.

In [5]:
df_movies_ = df_movies.drop(columns=['budget', 'homepage', 'imdb_id', 
                                    'original_title', 'poster_path', 
                                    'revenue', 'status', 'tagline', 
                                    'video'], axis=1)

In [6]:
pattern = '^[0-9]+$'
df_movies_['bool_Id'] = [bool(re.search(pattern, id_str)) for id_str in df_movies_['id']]

df_movies_ = df_movies_[df_movies_['bool_Id'] == True]
df_movies_.loc[:, 'id'] = df_movies_.loc[:, 'id'].astype(int)

Before merge datasets and extract genres, we need to confirm that movie `id` are unique:

In [7]:
try:
    assert len(df_movies_.id) == len(df_movies_.id.unique())
except:
    df_movies_.drop_duplicates(subset='id', inplace=True)

assert len(df_movies_.id) == len(df_movies_.id.unique())

Now, we extract the genres from the `genres` column. They are contained in lists of dictionaries.
How many id of `genres` I have on the dataset?

In [8]:
ind_genres = []
for genre in df_movies_.genres:
    elems = re.findall(r"'id': +[0-9]+", genre)
    for elem in elems:
        ind_genres.append(elem)

print('Number of different genres: {}'.format(len(np.unique(ind_genres))))
print('Corresponding to the following ids: {}'.format(np.unique(ind_genres)))

Number of different genres: 20
Corresponding to the following ids: ["'id': 10402" "'id': 10749" "'id': 10751" "'id': 10752" "'id': 10769"
 "'id': 10770" "'id': 12" "'id': 14" "'id': 16" "'id': 18" "'id': 27"
 "'id': 28" "'id': 35" "'id': 36" "'id': 37" "'id': 53" "'id': 80"
 "'id': 878" "'id': 9648" "'id': 99"]


Inspecting every genre:

In [9]:
ind = []
gen = []
counter = []

for idx, genre in zip(df_movies_.id, df_movies_.genres):
    elems = re.findall(r"'name': '+[a-zA-Z]+'", genre)
    n = len(elems)
    counter.append(n)
    for elem in elems:
        gen.append(elem)
        ind.append(idx)

print('Max number of genres per movie: {}'.format(np.max(counter)))
print('Number of different genres: {}'.format(len(np.unique(gen))))
print('Corresponding to the following ids: {}'.format(np.unique(gen)))

Max number of genres per movie: 8
Number of different genres: 18
Corresponding to the following ids: ["'name': 'Action'" "'name': 'Adventure'" "'name': 'Animation'"
 "'name': 'Comedy'" "'name': 'Crime'" "'name': 'Documentary'"
 "'name': 'Drama'" "'name': 'Family'" "'name': 'Fantasy'"
 "'name': 'Foreign'" "'name': 'History'" "'name': 'Horror'"
 "'name': 'Music'" "'name': 'Mystery'" "'name': 'Romance'"
 "'name': 'Thriller'" "'name': 'War'" "'name': 'Western'"]


For every id movie, we need 18 columns represented the list of possible genres. We create a dataFrame with `id_movies` and `name_genre`:

In [10]:
# Extracting only the name of the genre
name_genre = [g[9:-1] for g in gen]
# Building dataFrame with id_movies and genres
df_genre = pd.DataFrame({'id_movie': ind, 'genre': name_genre})
# Pivoting dataFrame to get 18 columns for every id_movie
df_genre = df_genre.pivot(index='id_movie', columns='genre', values='genre')
df_genre.head(2)

genre,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Thriller,War,Western
id_movie,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,Crime,NaN,Drama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Comedy,NaN,NaN,Drama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Merging with `df_movies_`:

In [11]:
df_movies_genres = df_movies_.merge(df_genre, left_on='id', right_on='id_movie').drop(columns='genres', axis=1)
df_movies_genres.head(2)

,adult,belongs_to_collection,id,original_language,overview,popularity,production_companies,production_countries,release_date,runtime,...,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Thriller,War,Western
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,NaN,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,104.0,...,Fantasy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Importing rating dataset as `df_rating`:

In [12]:
df_ratings = pd.read_csv('../Data/raw/movies/the-movies-dataset/ratings.csv', 
                        low_memory=False)

In [13]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
userId       int64
movieId      int64
rating       float64
timestamp    int64
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


In [14]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


`df_rating` is usefull if we want to build user profiles. For instance, to discover trend in users.

In [15]:
df_ratings[df_ratings['userId'] == 10]

,userId,movieId,rating,timestamp
401,10,150,4.0,846287850
402,10,153,3.0,846288047
403,10,165,4.0,846288047
404,10,296,5.0,846287850
405,10,344,4.0,846288047
406,10,349,5.0,846288047
407,10,380,5.0,846287850
408,10,457,4.0,846288177
409,10,588,5.0,846288047
410,10,590,3.0,846287850


Importing rating keywords as `df_keywords`:

In [16]:
df_keywords = pd.read_csv('../Data/raw/movies/the-movies-dataset/keywords.csv', 
                        low_memory=False)

In [17]:
df_keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
id          46419 non-null int64
keywords    46419 non-null object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [18]:
df_keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [19]:
key_list = []
for keyword in df_keywords.keywords:
    elems = re.findall(r"'name': '+[a-zA-Z]+'", keyword)
    k = []
    for elem in elems:
        elem = elem[9:-1]
        k.append(elem)
    key_list.append(k)  

In [20]:
df_keywords['key_list'] = key_list
df_keywords.head(2)

,id,keywords,key_list
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[disappearance, recluse]"


In [23]:
df_movies_genres_keywords = df_movies_genres.merge(df_keywords).drop(columns='keywords', axis=1)
df_movies_genres_keywords.head(2)

,adult,belongs_to_collection,id,original_language,overview,popularity,production_companies,production_countries,release_date,runtime,...,Foreign,History,Horror,Music,Mystery,Romance,Thriller,War,Western,key_list
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[jealousy, toy, boy, friendship, friends, riva..."
1,False,NaN,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,104.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[disappearance, recluse]"


In [24]:
#df_movies_genres_keywords.to_csv('../Data/interim/movies/movies.csv')